<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO


For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py


In [2]:
import datetime
#import keras
#from keras.datasets import mnist
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
#from keras.layers import Conv2D, MaxPooling2D
#from keras import backend as K
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [3]:
#used to help some of the timing functions
now = datetime.datetime.now

In [4]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [5]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [6]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [8]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [9]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [10]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [11]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [12]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [13]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [14]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples

Epoch 1/5


230/230 [==============================] - 12s 46ms/step - loss: 1.5985 - accuracy: 0.2474 - val_loss: 1.5784 - val_accuracy: 0.3662
Epoch 2/5
230/230 [==============================] - 10s 44ms/step - loss: 1.5758 - accuracy: 0.3043 - val_loss: 1.5507 - val_accuracy: 0.5289
Epoch 3/5
230/230 [==============================] - 10s 43ms/step - loss: 1.5485 - accuracy: 0.3793 - val_loss: 1.5206 - val_accuracy: 0.6095
Epoch 4/5
230/230 [==============================] - 10s 44ms/step - loss: 1.5213 - accuracy: 0.4432 - val_loss: 1.4876 - val_accuracy: 0.6651
Epoch 5/5
230/230 [==============================] - 12s 53ms/step - loss: 1.4897 - accuracy: 0.5027 - val_loss: 1.4508 - val_accuracy: 0.7021
Training time: 0:00:53.994513
Test score: 1.4507558345794678
Test accuracy: 0.7021189332008362


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.


In [15]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [17]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 6s 22ms/step - loss: 1.5684 - accuracy: 0.3180 - val_loss: 1.5347 - val_accuracy: 0.4622
Epoch 2/5
240/240 [==============================] - 6s 25ms/step - loss: 1.5210 - accuracy: 0.4120 - val_loss: 1.4843 - val_accuracy: 0.5657
Epoch 3/5
240/240 [==============================] - 5s 22ms/step - loss: 1.4760 - accuracy: 0.4987 - val_loss: 1.4335 - val_accuracy: 0.6811
Epoch 4/5
240/240 [==============================] - 6s 23ms/step - loss: 1.4285 - accuracy: 0.5728 - val_loss: 1.3820 - val_accuracy: 0.7690
Epoch 5/5
240/240 [==============================] - 5s 20ms/step - loss: 1.3829 - accuracy: 0.6341 - val_loss: 1.3305 - val_accuracy: 0.8332
Training time: 0:00:27.444152
Test score: 1.3304808139801025
Test accuracy: 0.833236038684845


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 


## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.


In [18]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [19]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 12s 46ms/step - loss: 1.6143 - accuracy: 0.2215 - val_loss: 1.5892 - val_accuracy: 0.3545
Epoch 2/5
240/240 [==============================] - 11s 48ms/step - loss: 1.5761 - accuracy: 0.3138 - val_loss: 1.5499 - val_accuracy: 0.4406
Epoch 3/5
240/240 [==============================] - 11s 44ms/step - loss: 1.5391 - accuracy: 0.3956 - val_loss: 1.5066 - val_accuracy: 0.5355
Epoch 4/5
240/240 [==============================] - 11s 44ms/step - loss: 1.4959 - accuracy: 0.4714 - val_loss: 1.4558 - val_accuracy: 0.6406
Epoch 5/5
240/240 [==============================] - 11s 45ms/step - loss: 1.4479 - accuracy: 0.5408 - val_loss: 1.3975 - val_accuracy: 0.7320
Training time: 0:00:55.309406
Test score: 1.3974976539611816
Test accuracy: 0.7320490479469299


In [20]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [22]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 5s 21ms/step - loss: 1.5781 - accuracy: 0.2787 - val_loss: 1.5509 - val_accuracy: 0.3458
Epoch 2/5
230/230 [==============================] - 5s 21ms/step - loss: 1.5478 - accuracy: 0.3287 - val_loss: 1.5184 - val_accuracy: 0.4088
Epoch 3/5
230/230 [==============================] - 5s 20ms/step - loss: 1.5187 - accuracy: 0.3804 - val_loss: 1.4865 - val_accuracy: 0.5104
Epoch 4/5
230/230 [==============================] - 6s 26ms/step - loss: 1.4903 - accuracy: 0.4335 - val_loss: 1.4552 - val_accuracy: 0.5900
Epoch 5/5
230/230 [==============================] - 5s 21ms/step - loss: 1.4608 - accuracy: 0.4903 - val_loss: 1.4241 - val_accuracy: 0.6521
Training time: 0:00:25.638216
Test score: 1.4241150617599487
Test accuracy: 0.6521291732788086


---
### Machine Learning Foundation (C) 2020 IBM Corporation
